In [3]:
!pip install -q keras

In [4]:
pip install split-folders[full]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import splitfolders
input_folder = '/content/drive/MyDrive/Scenes training set'
splitfolders.ratio(input_folder, output='dataset', seed= 42, ratio=(.7, 0, .3),
                   group_prefix=None)

Copying files: 14034 files [04:01, 58.11 files/s] 


In [6]:
%rm -rf '/content/drive/MyDrive/Scenes training set'

In [7]:
from keras_preprocessing.image.utils import np
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
#from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

IMAGE_SIZE = [224, 224]

train_path = "/content/dataset/train"
valid_path = "/content/dataset/test"


In [8]:
vgg= VGG19(input_shape=IMAGE_SIZE+[3], weights='imagenet', include_top=False)

for layer in vgg.layers:
  layer.trainable= False
folders = glob('dataset/train/*')

x= Flatten()(vgg.output)
prediction = Dense(len(folders), activation = 'softmax')(x)
model = Model (inputs=vgg.input, outputs= prediction)
model.summary()
model.compile(
    loss= 'categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)



80150528/80134624 [==============================] - 0s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                              

In [9]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen= ImageDataGenerator (rescale= 1./255,
                                   shear_range= 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255)
training_set = train_datagen.flow_from_directory('/content/dataset/train',
                                                 target_size = (224, 224),
                                                 batch_size= 32,
                                                 class_mode= 'categorical')
test_set = test_datagen.flow_from_directory('/content/dataset/test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 9820 images belonging to 6 classes.
Found 4214 images belonging to 6 classes.


In [10]:
r= model.fit_generator(
    training_set,
    validation_data=test_set,
    epochs=7,
    steps_per_epoch=len(training_set),
    validation_steps=len(test_set)

)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/7
307/307 [==============================] - 168s 507ms/step - loss: 0.6389 - accuracy: 0.7818 - val_loss: 0.3826 - val_accuracy: 0.8730
Epoch 2/7
307/307 [==============================] - 150s 488ms/step - loss: 0.4229 - accuracy: 0.8546 - val_loss: 0.4031 - val_accuracy: 0.8664
Epoch 3/7
307/307 [==============================] - 150s 487ms/step - loss: 0.3931 - accuracy: 0.8704 - val_loss: 0.3997 - val_accuracy: 0.8726
Epoch 4/7
307/307 [==============================] - 150s 487ms/step - loss: 0.3178 - accuracy: 0.8892 - val_loss: 0.4470 - val_accuracy: 0.8666
Epoch 5/7
307/307 [==============================] - 150s 487ms/step - loss: 0.3206 - accuracy: 0.8904 - val_loss: 0.4033 - val_accuracy: 0.8837
Epoch 6/7
307/307 [==============================] - 156s 508ms/step - loss: 0.3026 - accuracy: 0.8981 - val_loss: 0.4377 - val_accuracy: 0.8690
Epoch 7/7
307/307 [==============================] - 153s 498ms/step - loss: 0.3036 - accuracy: 0.8994 - val_loss: 0.4940 - val_ac

In [31]:
import os.path
model.save('models/vgg19model.h5')

In [109]:
from keras.models import load_model
from keras.preprocessing import image
import matplotlib.pyplot as plt
import numpy as np
import os
from os import listdir
from os.path import isfile, join
import numpy
import glob
import cv2 as cv


def load_image(img_path, show=False):

    img = image.load_img(img_path, target_size=(224, 224))
    img_tensor = image.img_to_array(img)                    # (height, width, channels)
    img_tensor = np.expand_dims(img_tensor, axis=0)         # (1, height, width, channels), add a dimension because the model expects this shape: (batch_size, height, width, channels)
    img_tensor /= 255.                                      # imshow expects values in the range [0, 1]

    if show:
        plt.imshow(img_tensor[0])                           
        plt.axis('off')
        plt.show()

    return img_tensor
classes_names=['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']




if __name__ == "__main__":

    # load model
    model = load_model("models/vgg19model.h5")

    # image path
    img_path = '/content/drive/MyDrive/Scenes testing test/103.jpg'    # dog
    #img_path = '/content/drive/MyDrive/Scenes testing test/103.jpg'      # cat

    # load a single image
    #new_image = load_image(img_path)


  
    path = glob.glob("/content/drive/MyDrive/Scenes testing test/*.jpg")
    cv_img = []
    predlab=[]
    imgname=[]
    for img in path:
       # n = cv.imread(img)
        n = load_image(img)
        cv_img.append(n)
        imgname.append(img)

for i in range (3050):
    # check prediction
    pred = model.predict(cv_img[i])
    ##print(img_path)
    print (np.argmax(pred))
    predlab.append(np.argmax(pred))

    ###################








4
3
1
4
4
1
3
4
4
4
2
4
4
3
3
4
4
4
4
3
3
5
4
4
4
4
3
4
4
3
4
4
4
0
3
2
5
4
4
5
4
4
4
4
4
3
4
4
4
4
1
4
4
1
4
4
4
4
4
4
4
4
4
4
4
4
4
4
2
4
4
4
4
4
4
4
4
4
4
2
4
4
4
2
4
4
4
4
4
4
4
5
4
2
4
4
4
4
4
4
3
4
0
2
2
4
4
4
4
4
1
4
4
4
4
4
4
5
4
4
4
4
2
4
4
4
4
4
4
4
4
4
4
1
4
4
4
2
4
4
4
4
4
4
4
4
4
4
1
2
0
4
3
4
2
4
4
3
5
4
4
4
4
4
4
4
4
4
4
4
4
2
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
3
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
2
4
4
4
4
3
4
3
3
4
4
4
4
4
3
2
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
1
4
4
4
4
4
4
4
4
4
4
2
4
4
4
4
4
4
4
4
4
4
4
4
5
4
4
4
4
4
4
4
4
4
4
4
3
4
3
4
4
4
4
0
4
4
4
4
4
4
4
4
4
4
0
4
4
4
0
4
3
4
4
4
4
4
4
4
4
4
4
4
5
4
0
4
4
4
4
4
4
4
4
4
4
4
4
1
4
4
4
4
4
4
4
4
4
4
2
3
5
4
4
4
4
2
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
1
5
4
4
4
4
4
4
4
4
4
4
4
4
4
4
0
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
2
4
4
4
5
4
4
4
4
4
4
4
4
4
4
4
3
4
4
4
4
4
4
4
4
2
2
3
4
4
4
4
4
4
4
3
4
4
4
4
4
4
4
4
4
4
3
4
4
4
4
4
4
4
2
4
5
4
5
5
5
5


In [110]:
from prettytable import PrettyTable
from prettytable import from_csv
import pandas as pd
import ntpath

table=PrettyTable(['Image','Label'])
for x in range(len(cv_img)):
  table.add_row([ntpath.basename(imgname[x]),predlab[x]])
print(table) 

+-----------+-------+
|   Image   | Label |
+-----------+-------+
| 20283.jpg |   4   |
| 24252.jpg |   3   |
| 20076.jpg |   1   |
| 20269.jpg |   4   |
| 20212.jpg |   4   |
| 20165.jpg |   1   |
| 24282.jpg |   3   |
| 20185.jpg |   4   |
| 20342.jpg |   4   |
| 20473.jpg |   4   |
| 20476.jpg |   2   |
| 20341.jpg |   4   |
| 20300.jpg |   4   |
| 24328.jpg |   3   |
| 24321.jpg |   3   |
| 20124.jpg |   4   |
| 20395.jpg |   4   |
| 20112.jpg |   4   |
| 20099.jpg |   4   |
| 24211.jpg |   3   |
| 24273.jpg |   3   |
| 20168.jpg |   5   |
| 20402.jpg |   4   |
| 20393.jpg |   4   |
| 20333.jpg |   4   |
| 20281.jpg |   4   |
| 24263.jpg |   3   |
| 20513.jpg |   4   |
| 20511.jpg |   4   |
| 24285.jpg |   3   |
| 20267.jpg |   4   |
| 20247.jpg |   4   |
| 20236.jpg |   4   |
| 20494.jpg |   0   |
| 24247.jpg |   3   |
| 24208.jpg |   2   |
| 20264.jpg |   5   |
| 20392.jpg |   4   |
| 20347.jpg |   4   |
| 20256.jpg |   5   |
| 20081.jpg |   4   |
| 20114.jpg |   4   |
| 20265.jp

In [115]:
from prettytable import PrettyTable
from prettytable import from_csv
import pandas as pd
import ntpath
namearr = []
for i in range (len(imgname)):
  namearr.append(ntpath.basename(imgname[i]))
for i in range (len(imgname)):
  print (namearr[i])

20283.jpg
24252.jpg
20076.jpg
20269.jpg
20212.jpg
20165.jpg
24282.jpg
20185.jpg
20342.jpg
20473.jpg
20476.jpg
20341.jpg
20300.jpg
24328.jpg
24321.jpg
20124.jpg
20395.jpg
20112.jpg
20099.jpg
24211.jpg
24273.jpg
20168.jpg
20402.jpg
20393.jpg
20333.jpg
20281.jpg
24263.jpg
20513.jpg
20511.jpg
24285.jpg
20267.jpg
20247.jpg
20236.jpg
20494.jpg
24247.jpg
24208.jpg
20264.jpg
20392.jpg
20347.jpg
20256.jpg
20081.jpg
20114.jpg
20265.jpg
20389.jpg
20273.jpg
24283.jpg
20170.jpg
20171.jpg
20672.jpg
20664.jpg
20622.jpg
20647.jpg
20531.jpg
20678.jpg
20530.jpg
20639.jpg
20649.jpg
20633.jpg
20520.jpg
20644.jpg
20634.jpg
20519.jpg
20679.jpg
20534.jpg
20595.jpg
20640.jpg
20822.jpg
20769.jpg
20873.jpg
20878.jpg
20759.jpg
20879.jpg
20722.jpg
20687.jpg
20834.jpg
20695.jpg
20753.jpg
20869.jpg
20696.jpg
20735.jpg
20692.jpg
20848.jpg
20841.jpg
20840.jpg
20856.jpg
20811.jpg
20704.jpg
20694.jpg
21442.jpg
21006.jpg
21217.jpg
20942.jpg
20882.jpg
21364.jpg
20975.jpg
21045.jpg
21405.jpg
21237.jpg
21258.jpg
21138.jpg


In [116]:
import pandas as pd
import numpy as np



df = pd.DataFrame({"Image" : namearr , "Label" : predlab })
df.to_csv("vgg19prediction.csv", index=False)